# Human Gene Length Database

## Downloading Data

In [ ]:
# Downloads and unzips basic annotations
!wget https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_36/gencode.v36.basic.annotation.gtf.gz
!gunzip gencode.v36.basic.annotation.gtf.gz

## Subsetting Genes, Transcripts, and Exons

In [ ]:
# Subsets genes, transcripts, and exons
!grep -P "\tgene\t" gencode.v36.basic.annotation.gtf > genes.gtf
!grep -P "\ttranscript\t" gencode.v36.basic.annotation.gtf > transcripts.gtf
!grep -P "\texon\t" gencode.v36.basic.annotation.gtf > exons.gtf

In [ ]:
# In the GTF format, columns are separated by tabs
# and the 9th column has several fields separated by ";"
# Genes, transcripts, and exons have different sets of 
# fields in the 9th column, so we need to process them
# individually

# Splits gene columns into two files for loading and re-merging
!cut -f1-8 genes.gtf > genes_basic.gtf
!cut -f9 genes.gtf > genes_extra.gtf
# Splits transcript columns into two files for loading and re-merging
!cut -f1-8 transcripts.gtf > transcripts_basic.gtf
!cut -f9 transcripts.gtf > transcripts_extra.gtf
# Splits exon columns into two files for loading and re-merging
!cut -f1-8 exons.gtf > exons_basic.gtf
!cut -f9 exons.gtf > exons_extra.gtf

## Loading Data into Pandas

In [1]:
import pandas as pd

def gen_df(feature):
    # Loads columns 1-8
    basic = pd.read_csv(f'./{feature}_basic.gtf', sep='\t', header=None, names=['chr', 'program', 'feature', 'start', 'end', 'score', 'strand', 'frame'])
    # Loads column 9, split into its fields
    extra = pd.read_csv(f'./{feature}_extra.gtf', header=None)
    # Builds a dictionary with the existing fields
    # and turns it into a DataFrame
    extra = extra.map(lambda v: dict([elem.strip().split(' ') for elem in v[:-1].split(';')]))
    extra = pd.DataFrame(list(extra.values.squeeze()))
    # Removes extra double-quotes from string fields
    extra = extra.map(lambda s: s[1:-1] if isinstance(s, str) and s[0] == '"' else s)
    # Merge them together
    df = pd.concat([basic, extra], axis=1).drop(columns=['score', 'frame'])
    # Compute length of the feature
    df['length'] = df['end'] - df['start'] + 1
    return df

In [12]:
# Transforms genes, transcripts, and exons files into DFs
genes = gen_df('genes')
transcripts = gen_df('transcripts')
exons = gen_df('exons')

In [3]:
genes

,chr,program,feature,start,end,strand,gene_id,gene_type,gene_name,level,hgnc_id,havana_gene,tag,length
0,chr1,HAVANA,gene,11869,14409,+,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,2,HGNC:37102,OTTHUMG00000000961.2,NaN,2541
1,chr1,HAVANA,gene,14404,29570,-,ENSG00000227232.5,unprocessed_pseudogene,WASH7P,2,HGNC:38034,OTTHUMG00000000958.1,NaN,15167
2,chr1,ENSEMBL,gene,17369,17436,-,ENSG00000278267.1,miRNA,MIR6859-1,3,HGNC:50039,NaN,NaN,68
3,chr1,HAVANA,gene,29554,31109,+,ENSG00000243485.5,lncRNA,MIR1302-2HG,2,HGNC:52482,OTTHUMG00000000959.2,ncRNA_host,1556
4,chr1,ENSEMBL,gene,30366,30503,+,ENSG00000284332.1,miRNA,MIR1302-2,3,HGNC:35294,NaN,NaN,138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60655,chrM,ENSEMBL,gene,14149,14673,-,ENSG00000198695.2,protein_coding,MT-ND6,3,HGNC:7462,NaN,NaN,525
60656,chrM,ENSEMBL,gene,14674,14742,-,ENSG00000210194.1,Mt_tRNA,MT-TE,3,HGNC:7479,NaN,NaN,69
60657,chrM,ENSEMBL,gene,14747,15887,+,ENSG00000198727.2,protein_coding,MT-CYB,3,HGNC:7427,NaN,NaN,1141
60658,chrM,ENSEMBL,gene,15888,15953,+,ENSG00000210195.2,Mt_tRNA,MT-TT,3,HGNC:7499,NaN,NaN,66


In [4]:
transcripts

,chr,program,feature,start,end,strand,gene_id,transcript_id,gene_type,gene_name,...,level,transcript_support_level,hgnc_id,tag,havana_gene,havana_transcript,ont,protein_id,ccdsid,length
0,chr1,HAVANA,transcript,11869,14409,+,ENSG00000223972.5,ENST00000456328.2,transcribed_unprocessed_pseudogene,DDX11L1,...,2,1,HGNC:37102,basic,OTTHUMG00000000961.2,OTTHUMT00000362751.1,NaN,NaN,NaN,2541
1,chr1,HAVANA,transcript,12010,13670,+,ENSG00000223972.5,ENST00000450305.2,transcribed_unprocessed_pseudogene,DDX11L1,...,2,NA,HGNC:37102,basic,OTTHUMG00000000961.2,OTTHUMT00000002844.2,PGO:0000019,NaN,NaN,1661
2,chr1,HAVANA,transcript,14404,29570,-,ENSG00000227232.5,ENST00000488147.1,unprocessed_pseudogene,WASH7P,...,2,NA,HGNC:38034,basic,OTTHUMG00000000958.1,OTTHUMT00000002839.1,PGO:0000005,NaN,NaN,15167
3,chr1,ENSEMBL,transcript,17369,17436,-,ENSG00000278267.1,ENST00000619216.1,miRNA,MIR6859-1,...,3,NA,HGNC:50039,basic,NaN,NaN,NaN,NaN,NaN,68
4,chr1,HAVANA,transcript,29554,31097,+,ENSG00000243485.5,ENST00000473358.1,lncRNA,MIR1302-2HG,...,2,5,HGNC:52482,basic,OTTHUMG00000000959.2,OTTHUMT00000002840.1,NaN,NaN,NaN,1544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110077,chrM,ENSEMBL,transcript,14149,14673,-,ENSG00000198695.2,ENST00000361681.2,protein_coding,MT-ND6,...,3,NA,HGNC:7462,appris_principal_1,NaN,NaN,NaN,ENSP00000354665.2,NaN,525
110078,chrM,ENSEMBL,transcript,14674,14742,-,ENSG00000210194.1,ENST00000387459.1,Mt_tRNA,MT-TE,...,3,NA,HGNC:7479,basic,NaN,NaN,NaN,NaN,NaN,69
110079,chrM,ENSEMBL,transcript,14747,15887,+,ENSG00000198727.2,ENST00000361789.2,protein_coding,MT-CYB,...,3,NA,HGNC:7427,appris_principal_1,NaN,NaN,NaN,ENSP00000354554.2,NaN,1141
110080,chrM,ENSEMBL,transcript,15888,15953,+,ENSG00000210195.2,ENST00000387460.2,Mt_tRNA,MT-TT,...,3,NA,HGNC:7499,basic,NaN,NaN,NaN,NaN,NaN,66


In [5]:
exons

,chr,program,feature,start,end,strand,gene_id,transcript_id,gene_type,gene_name,...,level,transcript_support_level,hgnc_id,tag,havana_gene,havana_transcript,ont,protein_id,ccdsid,length
0,chr1,HAVANA,exon,11869,12227,+,ENSG00000223972.5,ENST00000456328.2,transcribed_unprocessed_pseudogene,DDX11L1,...,2,1,HGNC:37102,basic,OTTHUMG00000000961.2,OTTHUMT00000362751.1,NaN,NaN,NaN,359
1,chr1,HAVANA,exon,12613,12721,+,ENSG00000223972.5,ENST00000456328.2,transcribed_unprocessed_pseudogene,DDX11L1,...,2,1,HGNC:37102,basic,OTTHUMG00000000961.2,OTTHUMT00000362751.1,NaN,NaN,NaN,109
2,chr1,HAVANA,exon,13221,14409,+,ENSG00000223972.5,ENST00000456328.2,transcribed_unprocessed_pseudogene,DDX11L1,...,2,1,HGNC:37102,basic,OTTHUMG00000000961.2,OTTHUMT00000362751.1,NaN,NaN,NaN,1189
3,chr1,HAVANA,exon,12010,12057,+,ENSG00000223972.5,ENST00000450305.2,transcribed_unprocessed_pseudogene,DDX11L1,...,2,NA,HGNC:37102,basic,OTTHUMG00000000961.2,OTTHUMT00000002844.2,PGO:0000019,NaN,NaN,48
4,chr1,HAVANA,exon,12179,12227,+,ENSG00000223972.5,ENST00000450305.2,transcribed_unprocessed_pseudogene,DDX11L1,...,2,NA,HGNC:37102,basic,OTTHUMG00000000961.2,OTTHUMT00000002844.2,PGO:0000019,NaN,NaN,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771718,chrM,ENSEMBL,exon,14149,14673,-,ENSG00000198695.2,ENST00000361681.2,protein_coding,MT-ND6,...,3,NA,HGNC:7462,appris_principal_1,NaN,NaN,NaN,ENSP00000354665.2,NaN,525
771719,chrM,ENSEMBL,exon,14674,14742,-,ENSG00000210194.1,ENST00000387459.1,Mt_tRNA,MT-TE,...,3,NA,HGNC:7479,basic,NaN,NaN,NaN,NaN,NaN,69
771720,chrM,ENSEMBL,exon,14747,15887,+,ENSG00000198727.2,ENST00000361789.2,protein_coding,MT-CYB,...,3,NA,HGNC:7427,appris_principal_1,NaN,NaN,NaN,ENSP00000354554.2,NaN,1141
771721,chrM,ENSEMBL,exon,15888,15953,+,ENSG00000210195.2,ENST00000387460.2,Mt_tRNA,MT-TT,...,3,NA,HGNC:7499,basic,NaN,NaN,NaN,NaN,NaN,66


## Sorting Data

In [13]:
# Creates a sortable chromosome info
exons['chr_sort'] = exons['chr'].apply(lambda v: v[:3]+'0'+v[-1] if v[-1].isnumeric() and len(v) == 4 else (v if v[-1] != 'M' else 'chr_M'))
# Sorts genes in the order they appear in chromosomes
ordered_genes = (exons
                 .groupby(['chr_sort', 'gene_id'])['start']
                 .min()
                 .reset_index()
                 .sort_values(by=['chr_sort', 'start'])
                 .reset_index(drop=True))

# Creates a sortable gene info
ordered_genes['gene_sort'] = list(map(lambda v: f'{(v+1):05d}', list(ordered_genes.index)))
exons = exons.merge(ordered_genes[['gene_id', 'gene_sort']], on='gene_id')

# Sorts transcripts in the order they appear in genes
ordered_tx = (exons
              .groupby(['chr_sort', 'gene_sort', 'transcript_id'])['start']
              .min()
              .reset_index()
              .sort_values(by=['chr_sort', 'gene_sort', 'start'])
              .reset_index(drop=True))
# Creates a sortable tx info
ordered_tx['tx_sort'] = list(map(lambda v: f'{(v+1):06d}', list(ordered_tx.index)))
exons = exons.merge(ordered_tx[['transcript_id', 'tx_sort']], on='transcript_id')

# Sorts exons by chromosome, gene, and transcript
exons = (exons
         .sort_values(by=['chr_sort', 'gene_sort', 'tx_sort', 'start'])
         [['chr', 'gene_id', 'transcript_id', 'exon_id', 'start', 'end', 'length']])

In [14]:
exons

,chr,gene_id,transcript_id,exon_id,start,end,length
0,chr1,ENSG00000223972.5,ENST00000456328.2,ENSE00002234944.1,11869,12227,359
1,chr1,ENSG00000223972.5,ENST00000456328.2,ENSE00003582793.1,12613,12721,109
2,chr1,ENSG00000223972.5,ENST00000456328.2,ENSE00002312635.1,13221,14409,1189
3,chr1,ENSG00000223972.5,ENST00000450305.2,ENSE00001948541.1,12010,12057,48
4,chr1,ENSG00000223972.5,ENST00000450305.2,ENSE00001671638.2,12179,12227,49
...,...,...,...,...,...,...,...
771718,chrM,ENSG00000198695.2,ENST00000361681.2,ENSE00001434974.2,14149,14673,525
771719,chrM,ENSG00000210194.1,ENST00000387459.1,ENSE00001544476.1,14674,14742,69
771720,chrM,ENSG00000198727.2,ENST00000361789.2,ENSE00001436074.2,14747,15887,1141
771721,chrM,ENSG00000210195.2,ENST00000387460.2,ENSE00001544475.2,15888,15953,66


## Adding Features

In [15]:
# Gets position, type, and length info for genes and transcripts
genes['gene_type'] = genes['gene_type'].map(lambda s: ' '.join(s.split('_')).capitalize())
gene_len = genes[['gene_id', 'start', 'gene_type', 'gene_name']].rename(columns={'start': 'gene_start'})
tx_len = transcripts[['transcript_id', 'length']].rename(columns={'length': 'tx_length'})
exons = (exons
         .merge(gene_len, on='gene_id')
         .merge(tx_len, on='transcript_id')
         [['chr', 'gene_id', 'gene_name', 'gene_type', 'gene_start',
           'transcript_id', 'tx_length', 'exon_id', 'start', 'end', 'length']])

In [16]:
exons

,chr,gene_id,gene_name,gene_type,gene_start,transcript_id,tx_length,exon_id,start,end,length
0,chr1,ENSG00000223972.5,DDX11L1,Transcribed unprocessed pseudogene,11869,ENST00000456328.2,2541,ENSE00002234944.1,11869,12227,359
1,chr1,ENSG00000223972.5,DDX11L1,Transcribed unprocessed pseudogene,11869,ENST00000456328.2,2541,ENSE00003582793.1,12613,12721,109
2,chr1,ENSG00000223972.5,DDX11L1,Transcribed unprocessed pseudogene,11869,ENST00000456328.2,2541,ENSE00002312635.1,13221,14409,1189
3,chr1,ENSG00000223972.5,DDX11L1,Transcribed unprocessed pseudogene,11869,ENST00000450305.2,1661,ENSE00001948541.1,12010,12057,48
4,chr1,ENSG00000223972.5,DDX11L1,Transcribed unprocessed pseudogene,11869,ENST00000450305.2,1661,ENSE00001671638.2,12179,12227,49
...,...,...,...,...,...,...,...,...,...,...,...
771718,chrM,ENSG00000198695.2,MT-ND6,Protein coding,14149,ENST00000361681.2,525,ENSE00001434974.2,14149,14673,525
771719,chrM,ENSG00000210194.1,MT-TE,Mt trna,14674,ENST00000387459.1,69,ENSE00001544476.1,14674,14742,69
771720,chrM,ENSG00000198727.2,MT-CYB,Protein coding,14747,ENST00000361789.2,1141,ENSE00001436074.2,14747,15887,1141
771721,chrM,ENSG00000210195.2,MT-TT,Mt trna,15888,ENST00000387460.2,66,ENSE00001544475.2,15888,15953,66


## Compute Lengths

In [17]:
def count_bp(row):
    starts = row['start']
    ends = row['end']
    start = min(starts)
    end = max(ends)
    bp = [False] * (end - start + 1)
    for i in range(len(starts)):
        s = starts[i] - start
        e = ends[i] - start + 1
        bp[s:e] = [True] * (e - s)
    return sum(bp)

# Computes length of merged transcripts
merged_len = (exons
              .groupby(['gene_id'])[['start', 'end']]
              .agg(list)
              .apply(count_bp, axis=1)
              .to_frame('merged_length')
              .reset_index())

# Computes length of each transcript (sum of exons)
transcript_len = exons.groupby(['gene_id', 'transcript_id'])['length'].sum()
# Computes average, maximum, and median lengths over all transcripts
avg_len = transcript_len.groupby(level=[0]).agg('mean').to_frame('mean_length').reset_index()
max_len = transcript_len.groupby(level=[0]).agg('max').to_frame('max_length').reset_index()
median_len = transcript_len.groupby(level=[0]).agg('median').to_frame('median_length').reset_index()

# Computes number of transcripts per gene
n_transcripts = (exons[['gene_id', 'transcript_id']]
                 .drop_duplicates()
                 .groupby(['gene_id'])['transcript_id']
                 .count()
                 .to_frame('n_tx')
                 .reset_index())

exons = (exons
         .merge(n_transcripts, on='gene_id')
         .merge(merged_len, on='gene_id')
         .merge(avg_len, on='gene_id')
         .merge(median_len, on='gene_id')
         .merge(max_len, on='gene_id'))

In [18]:
exons

,chr,gene_id,gene_name,gene_type,gene_start,transcript_id,tx_length,exon_id,start,end,length,n_tx,merged_length,mean_length,median_length,max_length
0,chr1,ENSG00000223972.5,DDX11L1,Transcribed unprocessed pseudogene,11869,ENST00000456328.2,2541,ENSE00002234944.1,11869,12227,359,2,1735,1144.5,1144.5,1657
1,chr1,ENSG00000223972.5,DDX11L1,Transcribed unprocessed pseudogene,11869,ENST00000456328.2,2541,ENSE00003582793.1,12613,12721,109,2,1735,1144.5,1144.5,1657
2,chr1,ENSG00000223972.5,DDX11L1,Transcribed unprocessed pseudogene,11869,ENST00000456328.2,2541,ENSE00002312635.1,13221,14409,1189,2,1735,1144.5,1144.5,1657
3,chr1,ENSG00000223972.5,DDX11L1,Transcribed unprocessed pseudogene,11869,ENST00000450305.2,1661,ENSE00001948541.1,12010,12057,48,2,1735,1144.5,1144.5,1657
4,chr1,ENSG00000223972.5,DDX11L1,Transcribed unprocessed pseudogene,11869,ENST00000450305.2,1661,ENSE00001671638.2,12179,12227,49,2,1735,1144.5,1144.5,1657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771718,chrM,ENSG00000198695.2,MT-ND6,Protein coding,14149,ENST00000361681.2,525,ENSE00001434974.2,14149,14673,525,1,525,525.0,525.0,525
771719,chrM,ENSG00000210194.1,MT-TE,Mt trna,14674,ENST00000387459.1,69,ENSE00001544476.1,14674,14742,69,1,69,69.0,69.0,69
771720,chrM,ENSG00000198727.2,MT-CYB,Protein coding,14747,ENST00000361789.2,1141,ENSE00001436074.2,14747,15887,1141,1,1141,1141.0,1141.0,1141
771721,chrM,ENSG00000210195.2,MT-TT,Mt trna,15888,ENST00000387460.2,66,ENSE00001544475.2,15888,15953,66,1,66,66.0,66.0,66


## Assembling and Saving Results

In [19]:
# Assembles final table
result = exons[['chr', 'gene_id', 'gene_name', 'gene_type', 'gene_start', 'n_tx', 'merged_length', 'mean_length', 'median_length', 'max_length']].drop_duplicates()
# Assigns sequential gene number within chromosome
result['seq'] = result.assign(seq=1).groupby(['chr'])['seq'].agg('cumsum')
# Turns mean and median lengths into integers
result['mean_length'] = result['mean_length'].astype(int)
result['median_length'] = result['median_length'].astype(int)

# Selects the columns in the right order
result = result[['seq', 'chr', 'gene_id', 'gene_name', 'gene_type', 'gene_start', 'n_tx', 'merged_length', 'mean_length', 'median_length', 'max_length']]

In [20]:
result

,seq,chr,gene_id,gene_name,gene_type,gene_start,n_tx,merged_length,mean_length,median_length,max_length
0,1,chr1,ENSG00000223972.5,DDX11L1,Transcribed unprocessed pseudogene,11869,2,1735,1144,1144,1657
9,2,chr1,ENSG00000227232.5,WASH7P,Unprocessed pseudogene,14404,1,1351,1351,1351,1351
20,3,chr1,ENSG00000278267.1,MIR6859-1,Mirna,17369,1,68,68,68,68
21,4,chr1,ENSG00000243485.5,MIR1302-2HG,Lncrna,29554,2,1021,623,623,712
26,5,chr1,ENSG00000284332.1,MIR1302-2,Mirna,30366,1,138,138,138,138
...,...,...,...,...,...,...,...,...,...,...,...
771718,33,chrM,ENSG00000198695.2,MT-ND6,Protein coding,14149,1,525,525,525,525
771719,34,chrM,ENSG00000210194.1,MT-TE,Mt trna,14674,1,69,69,69,69
771720,35,chrM,ENSG00000198727.2,MT-CYB,Protein coding,14747,1,1141,1141,1141,1141
771721,36,chrM,ENSG00000210195.2,MT-TT,Mt trna,15888,1,66,66,66,66


In [ ]:
result.to_csv('genes.csv', index=False)